# imports

In [ ]:
    import pandas as pd
    import requests
    import numpy as np
    import time
    import bs4
    from bs4 import BeautifulSoup
    import selenium
    from selenium import webdriver
    from selenium.webdriver.common.keys import Keys
    from selenium.webdriver.common.by import By
    from selenium.webdriver.chrome.service import Service
    from webdriver_manager.chrome import ChromeDriverManager
    from selenium.webdriver.support.select import Select
    from selenium.webdriver.common.action_chains import ActionChains
    

# population page extraction

In [ ]:
def extract_population(page,extract_page_catagory_name):
     # and returns it
    time.sleep(5)
    anti_exception=True
    error_counter=0
    while anti_exception==True and error_counter<5: #anti program failsafe 
        try:
            temp = page.find_element(By.XPATH, "/html/body/form/div[3]/div/div/div/div[4]/table/tbody/tr[2]")
            anti_exception=False #will pass the temp and not get caught when 
        except:
            print("failed to retrieve page element error number:", error_counter)
            error_counter=error_counter+1
            
            
    col_names = temp.find_elements(By.TAG_NAME, "th")
    col_names_list = ["סמל יישוב","שם יישוב"]
    for col_name in col_names:  # get columns names
        col_names_list.append(col_name.text)
    col_names_list[6]=col_names_list[6] + "זכר" #columns are seperatred between female and male
    col_names_list[7]=col_names_list[7] + "זכר" #its location on the page is mixed with some columns names
    col_names_list[8]=col_names_list[8] + "זכר" #which made it problemetic to extract 
    col_names_list[9]=col_names_list[9] + "נקבה"
    col_names_list[10]=col_names_list[10] + "נקבה"
    col_names_list[11]=col_names_list[11] + "נקבה"
    df = pd.DataFrame(columns=col_names_list)
    table = page.find_element(By.XPATH, "/html/body/form/div[3]/div/div/div/div[4]/table/tbody")  # get table
    rows = table.find_elements(By.TAG_NAME, "tr")  # extract each row from a table
    table_len = len(rows)  #
    for row in rows[2:]:
        elements = row.find_elements(By.TAG_NAME, "td")  # extract each element from a row
        elements_text = []
        for i in elements:  # go through each cell in a row (as element not text)
            elements_text.append(i.text)  # convert element which is 1 cell in a row to text
        df.loc[
            len(df)] = elements_text  # adds a new line to the end of data frame len(df)return the length of the dataframe
    return df.copy()

# construction page extraction

In [ ]:
def extract_page_construction(page, extract_page_catagory_name):
    time.sleep(3)
    temp = page.find_element(By.XPATH, "/html/body/form/div[3]/div/div/div/div[4]/table/tbody/tr[2]")
    col_names = temp.find_elements(By.TAG_NAME, "th")
    col_names_list = []
    for col_name in col_names:  # get columns names
        col_names_list.append(col_name.text)
    df = pd.DataFrame(columns=col_names_list)
    table = page.find_element(By.XPATH, "/html/body/form/div[3]/div/div/div/div[4]/table/tbody")  # get table
    rows = table.find_elements(By.TAG_NAME, "tr")  # extract each row from a table
    for row in rows[2:]:
        elements = row.find_elements(By.TAG_NAME, "td")  # extract each element from a row
        elements_text = []
        for i in elements:  # go through each cell in a row (as element not text)
            elements_text.append(i.text)  # convert element which is 1 cell in a row to text
        df.loc[
            len(df)] = elements_text  # adds a new line to the end of data frame len(df)return the length of the dataframe
    return df.copy()

# general page extraction

In [ ]:
def extract_page(page, extract_page_catagory_name):  # creates new dataframe (including columns names) from the page
    # and returns it
    print("extract_page:", extract_page_catagory_name)
    if(extract_page_catagory_name=="construction"):
        return extract_page_construction(page, extract_page_catagory_name)
    if extract_page_catagory_name in ["information", "transportation", "construction", "social_economic index"]:  # columns location differ between catagories
        time.sleep(3)
        temp = page.find_element(By.XPATH, "/html/body/form/div[3]/div/div/div/div[4]/table/thead/tr")
        col_names = temp.find_elements(By.TAG_NAME, "th")
    if(extract_page_catagory_name=="population"):
        print("in population")
        return extract_population(page,extract_page_catagory_name)
    col_names_list = []
    for col_name in col_names:  # get columns names
        col_names_list.append(col_name.text)
    df = pd.DataFrame(columns=col_names_list)
    table = page.find_element(By.XPATH, "/html/body/form/div[3]/div/div/div/div[4]/table/tbody")  # get table
    rows = table.find_elements(By.TAG_NAME, "tr")  # extract each row from a table
    table_len = len(rows)
    for row in rows:
        elements = row.find_elements(By.TAG_NAME, "td")  # extract each element from a row
        elements_text = []
        for i in elements:  # go through each cell in a row (as element not text)
            elements_text.append(i.text)  # convert element which is 1 cell in a row to text
        print(elements_text)
        df.loc[
            len(df)] = elements_text  # adds a new line to the end of data frame len(df)return the length of the dataframe
    
    return df.copy()

# sub function to control past year population pages

In [ ]:
def extract_population_by_year(driver,year):#name pending
    time.sleep(3)
    #from here we extract population
    for i in range(3):#move catagories untill population catagory is reached
        time.sleep(2)
        catagory_menu = driver.find_element(By.XPATH, "/html/body/form/div[3]/div/div/div/div[2]/span[2]/div")
        catagory_menu.click()
        actions = ActionChains(driver)
        actions.send_keys(Keys.ARROW_DOWN)
        actions.send_keys(Keys.ENTER)
        actions.perform()
        
    for index in range(2021-year):#change the year to 2018
        time.sleep(1)
        year_menu = driver.find_element(By.XPATH, "/html/body/form/div[3]/div/div/div/div[2]/span[4]/div")
        year_menu.click()
        actions.send_keys(Keys.ARROW_DOWN)
        actions.send_keys(Keys.ENTER)
        actions.perform()
    pop=get_catagory(driver, 1, "population")# extract dataframe and send back
    
    
    return pop

# catagory retrievel function

In [ ]:
def get_catagory(pages, page_number, catagory_name="None"): # default values
    print(catagory_name)
    catagory_dataframe = pd.DataFrame()
    time.sleep(3)
    click_array = driver.find_element(By.XPATH, "/html/body/form/div[3]/div/div/div/div[5]/pager/div")  # contains
    # the pages and some more clickable
    pages = click_array.find_elements(By.TAG_NAME, "li")#collect all clickable elements that are pages in 1 list
    if page_number == 1:  # starting condition
        catagory_dataframe = extract_page(driver, catagory_name).copy()
        page_number = page_number + 1 
    for page in pages:#go through the element list that contains all elements that are pages
        if page.text == str(page_number):
            print("scanning page:", page_number)
            page.click() #click the page so its loads in selenium
            catagory_dataframe = pd.concat([catagory_dataframe, extract_page(driver, catagory_name)],
                                           ignore_index=True).copy() #sends the page that is loaded to other function to extract the info from it and link it to the previous dataframe
            page_number = page_number + 1
        if(page.get_attribute("class")=="nextPages"): #this one works removed for since second method is easier on the eye\n",
                page.click()
                catagory_dataframe=pd.concat([catagory_dataframe, get_catagory(driver,page_number,catagory_name)]) 
    return catagory_dataframe.copy()



# main function 2017 catagory extraction

In [ ]:

url = 'https://boardsgenerator.cbs.gov.il/pages/webparts/yishuvimpage.aspx'

PATH = 'C:\Program Files (x86)\chromedriver.exe'
driver = webdriver.Chrome()
driver.get(url)
time.sleep(5)
dataframe_catagory_list = []
target_year = 2018
catagory_names = ["information", "transportation", "construction", "population", "health", "mifkad2008",
                  "data_from_local_authorities", "salary", "welfare", "education", "water", "waste",
                  "social_economic index"]#the catagories in the page

i = 0
for category in catagory_names:#goes through all catagories in the list in order
    anti_exception=True
    error_counter=0
    print(category)
    if(category not in ["information", "transportation", "construction", "population","social_economic index"]):#the catagory is not what we wanted
        catagory_menu = driver.find_element(By.XPATH, "/html/body/form/div[3]/div/div/div/div[2]/span[2]/div")
        catagory_menu.click()
        actions = ActionChains(driver)
        actions.send_keys(Keys.ARROW_DOWN)
        actions.send_keys(Keys.ENTER)
        actions.perform()
        continue
    time.sleep(3)
    print("Scanning: ", category)
    actions = ActionChains(driver)
    if(category !="construction"):
        while anti_exception==True and error_counter<5: #anti crash failsafe 
            try:
                temp = driver.find_element(By.XPATH, "/html/body/form/div[3]/div/div/div/div[4]/table/tbody/tr[2]")
                year_menu = driver.find_element(By.XPATH, "/html/body/form/div[3]/div/div/div/div[2]/span[4]/div")
                anti_exception=False #if failed loading the element it wont reach here,it will reach only when no error was found loading the element
            except:
                print("failed to retrieve year menu element error number:", error_counter)
                error_counter=error_counter+1
    if category == "social_economic index":#change the year to 2017
        year_menu.click()
        actions.send_keys(Keys.ARROW_DOWN)
        actions.send_keys(Keys.ENTER)
        actions.perform()
        time.sleep(3)
    elif category == "salary":#change the year to 2017
        for index in range(2):
            year_menu.click()
            actions.send_keys(Keys.ARROW_DOWN)
            actions.send_keys(Keys.ENTER)
            actions.perform()
            time.sleep(3)
    if category == "information" or category == "transportation" or category == "population" :
        for index in range(2022 - target_year):#change the year to 2017
            year_menu.click()
            actions.send_keys(Keys.ARROW_DOWN)
            actions.send_keys(Keys.ENTER)
            actions.perform()
            time.sleep(3)
    dataframe_catagory_list.append(
        get_catagory(driver, 53, category))  # extract the all the pages in the catagory and send the catagory name 
    #and add to a list of dataframes
    #path = "C:\\Users\\omri\\Desktop\\" #C:\Users\ONE1\Desktop\omri
    try:#instead of changing each time,try to save to all 3 known locations
        dataframe_catagory_list[i].to_csv("C:\\Users\\omri\\Desktop\\" + category + ".csv", index=False,
                                      encoding='utf-8-sig') #my main pc
    except:
        try:
            dataframe_catagory_list[i].to_csv("C:\\Users\\ONE1\\Desktop\\omri\\" + category + ".csv", index=False,
                                      encoding='utf-8-sig')#my laptop
        except:
            try:
                dataframe_catagory_list[i].to_csv("C:\\Users\\ONE1\\Desktop\\omri\\" + category + ".csv", index=False,
                                      encoding='utf-8-sig') # nir change this to where you want to save
            except:
                raise # raise error if cant save
          
        #delete below this is a test
    catagory_menu = driver.find_element(By.XPATH, "/html/body/form/div[3]/div/div/div/div[2]/span[2]/div")
    catagory_menu.click()
    actions = ActionChains(driver)
    actions.send_keys(Keys.ARROW_DOWN)
    actions.send_keys(Keys.ENTER)
    actions.perform()
    i = i + 1
driver.quit()
time.sleep(5)
modular_population_extractor()

# population 2013-2016 extraction


In [ ]:
def modular_population_extractor():
    url = 'https://boardsgenerator.cbs.gov.il/pages/webparts/yishuvimpage.aspx'
    path = 'C:\Program Files (x86)\chromedriver.exe'
    year_array=[2018,2017,2016,2015,2014,2013] #add year 
    file_path='C:\\Users\\ONE1\\Desktop\\omri\\'  #C:\Users\ONE1\Desktop\omri
    for pop_year in year_array:
        print("loading year:",pop_year)
        time.sleep(3)
        driver = webdriver.Chrome()
        driver.get(url)
        pop_dataframe=extract_population_by_year(driver,pop_year)
        pop_dataframe.to_csv(file_path + "pop" + str(pop_year) + ".csv", index=False,encoding='utf-8-sig')
    driver.quit()
